---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

## My Stuff

This is where I am going to poke around and play with the data before building the model for the answer.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
us_states  = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
             'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
             'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
             'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
disposition_replace = {
    'Responsible by Default': 'By default',
    'Responsible by Determination': 'By determination', 
    'Responsible (Fine Waived) by Deter': 'Fine Waived',
    'Responsible by Admission': 'By admission',
    'SET-ASIDE (PENDING JUDGMENT)': 'Pending',
    'PENDING JUDGMENT': 'Pending',
    'Not responsible by Dismissal': 'Not responsible',
    'Not responsible by City Dismissal': 'Not responsible',
    'Not responsible by Determination': 'Not responsible',
    'Responsible (Fine Waived) by Admis': 'Fine Waived',
    'Responsible - Compl/Adj by Default': 'By default',
    'Responsible - Compl/Adj by Determi': 'By determination',
    'Responsible by Dismissal': 'By default'
}

def clean(df):
    # Collapse state data into in-state or out of state (or out of country)
    df['in_state'] = df.state.apply(lambda x: 'yes' if x=='MI' else ('no' if x in us_states else 'foreign'))
    
    # Clean disposition values
    df.disposition.replace(disposition_replace, inplace=True)
    
    # These columns just aren't useful to us
    df.drop([
        'violation_zip_code', 
        'non_us_str_code', 
        'grafitti_status', 
        'violator_name', 
        'violation_street_number', 
        'violation_street_name', 
        'violation_description',
        'mailing_address_str_number', 
        'mailing_address_str_name',
        'city',
        'state',
        'zip_code',
        'country',
        'clean_up_cost'
    ], axis=1, inplace=True)
    
    # Date columns
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
    df['hearing_date'] = pd.to_datetime(df['hearing_date'])
    df['hearing_date'].fillna((df.hearing_date - df.hearing_date.min()).mean() + df.hearing_date.min(), inplace=True)
    df['ticket_issued_date'].fillna((df.ticket_issued_date - df.ticket_issued_date.min()).mean() + df.ticket_issued_date.min(), inplace=True)
    df['date_gap'] = ((df['hearing_date'] - df['ticket_issued_date'])/np.timedelta64(1, 'D')).astype(int)
    
    # Set index
    df.set_index('ticket_id', inplace=True)
    
    return df

In [3]:
df = clean(pd.read_csv('train.csv'))

# These rows aren't useful
df.dropna(subset=['compliance'], inplace=True)
    
# Categorical data
df.compliance = df.compliance.astype("category")

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3013 entries, 22056 to 22302
Data columns (total 21 columns):
agency_name           3013 non-null object
inspector_name        3013 non-null object
ticket_issued_date    3013 non-null datetime64[ns]
hearing_date          3013 non-null datetime64[ns]
violation_code        3013 non-null object
disposition           3013 non-null object
fine_amount           3013 non-null float64
admin_fee             3013 non-null float64
state_fee             3013 non-null float64
late_fee              3013 non-null float64
discount_amount       3013 non-null float64
judgment_amount       3013 non-null float64
payment_amount        3013 non-null float64
balance_due           3013 non-null float64
payment_date          1163 non-null object
payment_status        3013 non-null object
collection_status     629 non-null object
compliance_detail     3013 non-null object
compliance            3013 non-null category
in_state              3013 non-null object
dat

In [4]:
df.head()

,agency_name,inspector_name,ticket_issued_date,hearing_date,violation_code,disposition,fine_amount,admin_fee,state_fee,late_fee,...,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,compliance_detail,compliance,in_state,date_gap
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",2004-03-16 11:40:00,2005-03-21 10:30:00,9-1-36(a),By default,250.0,20.0,10.0,25.0,...,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,non-compliant by no payment,0.0,no,369
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin",2004-04-23 12:30:00,2005-05-06 13:30:00,61-63.0600,By determination,750.0,20.0,10.0,75.0,...,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,compliant by late payment within 1 month,1.0,yes,378
22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",2004-05-01 11:50:00,2005-03-21 10:30:00,9-1-36(a),By default,250.0,20.0,10.0,25.0,...,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,non-compliant by no payment,0.0,no,323
18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin",2004-06-14 14:15:00,2005-02-22 15:00:00,61-63.0500,By default,750.0,20.0,10.0,75.0,...,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,non-compliant by no payment,0.0,yes,253
18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin",2004-06-16 12:30:00,2005-02-22 15:00:00,61-63.0100,By default,100.0,20.0,10.0,10.0,...,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,non-compliant by no payment,0.0,yes,251


In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

gnb = GaussianNB()

fine_cols_subset = [
    'fine_amount', 
    'late_fee',
    'discount_amount'
]

X_train, X_test, y_train, y_test = train_test_split(df[fine_cols_subset], df['compliance'])
gnb.fit(X_train, y_train)
gnb.score(X_test, y_test)

0.9058355437665783

That score looks good enough to me! I have to use the AUC scoring function, though, not just accuracy.

In [6]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, gnb.predict(X_test))

0.5069444444444444

Oh, hmm. That's bad actually. Let's see that confusion matrix.

In [7]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, gnb.predict(X_test))

array([[682,   0],
       [ 71,   1]])

Looks like we are predicting a lot less compliance than there are.

Maybe a support vector machine could be better at this?

In [8]:
from sklearn.svm import SVC
svc = SVC().fit(X_train, y_train)
roc_auc_score(y_test, svc.predict(X_test))

0.5069444444444444

Also bad. Wow.

In [9]:
confusion_matrix(y_test, svc.predict(X_test))

array([[682,   0],
       [ 71,   1]])

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50).fit(X_train, y_train)
roc_auc_score(y_test, rf.predict(X_test))

0.5069444444444444

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression().fit(X_train, y_train)
roc_auc_score(y_test, lr.predict(X_test))

0.5069444444444444

Maybe I should be using the addresses. I didn't think I should use them because I... I don't know why. I'll just use them.

*Later edit* They didn't really help, so I continued to not use them.

In [12]:
# addrlatlon = pd.merge(pd.read_csv('addresses.csv'), pd.read_csv('latlons.csv'), on='address')
# train_df_new = (pd.merge(train_df, addrlatlon, on='ticket_id')
#                 .set_index('ticket_id')
#                 .drop(['violation_street_number', 'violation_street_name', 'address'], axis=1))
# train_df_new.head()

In [13]:
# new_cols = fine_cols_subset + ['lat', 'lon']
# new_df_to_train = train_df_new[new_cols + ['compliance']]
# X_train, X_test, y_train, y_test = train_test_split(new_df_to_train[new_cols], new_df_to_train['compliance'])
# new_model = GaussianNB().fit(X_train, y_train)
# roc_auc_score(y_test, new_model.predict(X_test))

Ok, it got worse. This is not great.

What am I even comparing to. 50% ROC AUC seems pretty bad. What do I get as a baseline of guessing all non-compliance?

In [14]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
roc_auc_score(y_test, dummy.predict(X_test))

0.5

In [15]:
confusion_matrix(y_test, dummy.predict(X_test))

array([[682,   0],
       [ 72,   0]])

So 50% is the absolute minimum performance. And all my models so far have been barely above that.

I think it is time to engineer some features, and/or turn these numerical features into categories.

Taking inspiration from https://www.kaggle.com/kylingu/apply-machine-learning-module-4-assignment

In [16]:
def make_features(df):
    dummy = pd.get_dummies(df[['in_state', 'disposition', 'agency_name']])

    # Make sure all the categorical columns got created, even if they're blank
    # This ensures training and test data have the same columns
    for cat_col in ['in_state_yes', 'in_state_no', 'in_state_foreign',
                    'disposition_By admission', 'disposition_By default', 'disposition_By determination', 'disposition_Fine Waived',
                    'agency_name_Buildings, Safety Engineering & Env Department',
                    'agency_name_Department of Public Works',
                    'agency_name_Detroit Police Department',
                    'agency_name_Health Department'
                   ]:
        if not cat_col in dummy.columns:
            dummy[cat_col] = 0

    return pd.concat(
        (df[['discount_amount', 'judgment_amount', 'date_gap']], 
         dummy), 
        axis=1)

In [17]:
y_train = df['compliance']
X_train = make_features(df)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3013 entries, 22056 to 22302
Data columns (total 14 columns):
discount_amount                                               3013 non-null float64
judgment_amount                                               3013 non-null float64
date_gap                                                      3013 non-null int64
in_state_no                                                   3013 non-null uint8
in_state_yes                                                  3013 non-null uint8
disposition_By admission                                      3013 non-null uint8
disposition_By default                                        3013 non-null uint8
disposition_By determination                                  3013 non-null uint8
agency_name_Buildings, Safety Engineering & Env Department    3013 non-null uint8
agency_name_Department of Public Works                        3013 non-null uint8
agency_name_Detroit Police Department                         30

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()
gs = GridSearchCV(rf, {'n_estimators': range(70, 150, 3)}, scoring='roc_auc', cv=5).fit(X_train, y_train)
# gs = GridSearchCV(rf, {'n_estimators': range(1, 50, 3)}, scoring='roc_auc', cv=5).fit(features_df, train_df['compliance'])

print("Best score: {}".format(gs.best_score_))
print("Best params: {}".format(gs.best_params_))

Best score: 0.6282218174735147
Best params: {'n_estimators': 100}


In [19]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()
gs = (GridSearchCV(gbc, 
                   {'learning_rate': [0.005, 0.1, 0.3, 1],
                    'max_depth':[2, 3, 5]}, 
                   scoring='roc_auc', 
                   cv=5)
      .fit(X_train, y_train))
#       .fit(features_df, train_df['compliance']))

print("Best score: {}".format(gs.best_score_))
print("Best params: {}".format(gs.best_params_))

Best score: 0.7097359048857025
Best params: {'learning_rate': 0.1, 'max_depth': 3}


That's over the 70% threshold I need to get for the test set. ...but it's with the training set, and I suspect the score will be worse for the test set.

In [20]:
X_test = make_features(clean(pd.read_csv('test.csv')))

X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 14 columns):
discount_amount                                               61001 non-null float64
judgment_amount                                               61001 non-null float64
date_gap                                                      61001 non-null int64
in_state_foreign                                              61001 non-null uint8
in_state_no                                                   61001 non-null uint8
in_state_yes                                                  61001 non-null uint8
disposition_By admission                                      61001 non-null uint8
disposition_By default                                        61001 non-null uint8
disposition_By determination                                  61001 non-null uint8
disposition_Fine Waived                                       61001 non-null uint8
agency_name_Buildings, Safety Engineering & Env Dep

In [21]:
X_test['compliance'] = gs.predict_proba(X_test)[:, 0]
X_test['compliance']

ticket_id
284932    0.698134
285362    0.548315
285361    0.756617
285338    0.846667
285346    0.560397
285345    0.644350
285347    0.936386
285342    0.965349
285530    0.176781
284989    0.313517
285344    0.936386
285343    0.548315
285340    0.548315
285341    0.936386
285349    0.289185
285348    0.401311
284991    0.313517
285532    0.366832
285406    0.459083
285001    0.528260
285006    0.285294
285405    0.508486
285337    0.687929
285496    0.931806
285497    0.846667
285378    0.327270
285589    0.407686
285585    0.650391
285501    0.756617
285581    0.836061
            ...   
376367    0.574087
376366    0.579980
376362    0.681730
376363    0.839088
376365    0.574087
376364    0.579980
376228    0.484021
376265    0.484021
376286    0.859793
376320    0.812271
376314    0.812271
376327    0.923484
376385    0.953665
376435    0.808541
376370    0.882576
376434    0.855808
376459    0.817031
376478    0.418009
376473    0.703020
376484    0.586257
376482    0.718740
37

All right. That's what I need to return from my function. I can't submit it, since I didn't pay for the class. That means I don't have the actual compliance for the test data and I won't know what the actual score is. But I gave it my best. I meaningfully improved the score through feature engineering. So... good I guess?

## Actual code for submission

In [22]:
def blight_model():
    import pandas as pd
    import numpy as np
    
    us_states  = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
                 'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
                 'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
                 'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
    disposition_replace = {
        'Responsible by Default': 'By default',
        'Responsible by Determination': 'By determination', 
        'Responsible (Fine Waived) by Deter': 'Fine Waived',
        'Responsible by Admission': 'By admission',
        'SET-ASIDE (PENDING JUDGMENT)': 'Pending',
        'PENDING JUDGMENT': 'Pending',
        'Not responsible by Dismissal': 'Not responsible',
        'Not responsible by City Dismissal': 'Not responsible',
        'Not responsible by Determination': 'Not responsible',
        'Responsible (Fine Waived) by Admis': 'Fine Waived',
        'Responsible - Compl/Adj by Default': 'By default',
        'Responsible - Compl/Adj by Determi': 'By determination',
        'Responsible by Dismissal': 'By default'
    }

    def clean(df):
        # Collapse state data into in-state or out of state (or out of country)
        df['in_state'] = df.state.apply(lambda x: 'yes' if x=='MI' else ('no' if x in us_states else 'foreign'))

        # Clean disposition values
        df.disposition.replace(disposition_replace, inplace=True)

        # These columns just aren't useful to us
        df.drop([
            'violation_zip_code', 
            'non_us_str_code', 
            'grafitti_status', 
            'violator_name', 
            'violation_street_number', 
            'violation_street_name', 
            'violation_description',
            'mailing_address_str_number', 
            'mailing_address_str_name',
            'city',
            'state',
            'zip_code',
            'country',
            'clean_up_cost'
        ], axis=1, inplace=True)

        # Date columns
        df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
        df['hearing_date'] = pd.to_datetime(df['hearing_date'])
        df['hearing_date'].fillna((df.hearing_date - df.hearing_date.min()).mean() + df.hearing_date.min(), inplace=True)
        df['ticket_issued_date'].fillna((df.ticket_issued_date - df.ticket_issued_date.min()).mean() + df.ticket_issued_date.min(), inplace=True)
        df['date_gap'] = ((df['hearing_date'] - df['ticket_issued_date'])/np.timedelta64(1, 'D')).astype(int)

        # Set index
        df.set_index('ticket_id', inplace=True)

        return df
    
    def make_features(df):
        dummy = pd.get_dummies(df[['in_state', 'disposition', 'agency_name']])

        # Make sure all the categorical columns got created, even if they're blank
        # This ensures training and test data have the same columns
        for cat_col in ['in_state_yes', 'in_state_no', 'in_state_foreign',
                        'disposition_By admission', 'disposition_By default', 'disposition_By determination', 'disposition_Fine Waived',
                        'agency_name_Buildings, Safety Engineering & Env Department',
                        'agency_name_Department of Public Works',
                        'agency_name_Detroit Police Department',
                        'agency_name_Health Department'
                       ]:
            if not cat_col in dummy.columns:
                dummy[cat_col] = 0

        return pd.concat(
            (df[['discount_amount', 'judgment_amount', 'date_gap']], 
             dummy), 
            axis=1)
    
    df = clean(pd.read_csv('train.csv'))
    df.dropna(subset=['compliance'], inplace=True)
    df.compliance = df.compliance.astype("category")
    y_train = df['compliance']
    X_train = make_features(df)
    
    gbc = GradientBoostingClassifier(learning_rate=0.1, max_depth=3).fit(X_train, y_train)
    
    X_test = make_features(clean(pd.read_csv('test.csv')))
    X_test['compliance'] = gs.predict_proba(X_test)[:, 0].astype('float32')
    
    return X_test['compliance']

In [23]:
blight_model()

ticket_id
284932    0.698134
285362    0.548315
285361    0.756617
285338    0.846667
285346    0.560397
285345    0.644350
285347    0.936386
285342    0.965349
285530    0.176781
284989    0.313517
285344    0.936386
285343    0.548315
285340    0.548315
285341    0.936386
285349    0.289185
285348    0.401311
284991    0.313517
285532    0.366832
285406    0.459083
285001    0.528260
285006    0.285294
285405    0.508486
285337    0.687929
285496    0.931806
285497    0.846667
285378    0.327270
285589    0.407686
285585    0.650391
285501    0.756617
285581    0.836061
            ...   
376367    0.574087
376366    0.579980
376362    0.681730
376363    0.839088
376365    0.574087
376364    0.579980
376228    0.484021
376265    0.484021
376286    0.859793
376320    0.812271
376314    0.812271
376327    0.923484
376385    0.953665
376435    0.808541
376370    0.882576
376434    0.855808
376459    0.817031
376478    0.418009
376473    0.703020
376484    0.586257
376482    0.718740
37